In [1]:
import script
from flask import Flask, jsonify, request
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json


In [2]:
app = Flask(__name__)

In [3]:
@app.route('/api/machine_learning', methods = ['GET'])
def get_machine_learning():
    args = request.args
    id = args.get("id")
    recipe_list = script.main_function(int(id))
    return {'recipe_list': recipe_list}, 200


In [4]:
@app.route('/api/tags', methods = ['GET'])
def get_tags():
    smallRecipes = pd.read_csv('../src/recipesDB/Small_Recipes.csv')
    def returnSteps(smallRecipe):
        smallRecipe = pd.DataFrame(smallRecipe.set_index('id')['tags'])
        smallRecipe["tags"] = smallRecipe["tags"].str.replace("[", '')
        smallRecipe["tags"] = smallRecipe["tags"].str.replace("']", '')
        smallRecipe = smallRecipe.tags.str.split("', '",expand=True)
        smallRecipe[0] = smallRecipe[0].str.replace("'", '')
        return smallRecipe
    allSteps = returnSteps(smallRecipes)
    tags = []
    for i in range(len(allSteps.columns)):
        tags = tags + list(allSteps.iloc[:,i].unique())
    tags = set(tags)
    while '' in tags:
        tags.remove('')
    while None in tags:
        tags.remove(None)
    tags = list(tags)

    return {'tags': tags}, 200
    

In [5]:
@app.route('/api/recherche', methods = ['GET'])
def get_recherche_nom():
    args = request.args
    name_tags = args.get("search_bar")
    tags = args.get("search_button")
    result = []
    df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
    if name_tags is not None:
        df1 = df[["name","id"]].copy()
        df1 = df1.set_index("id")
        df1 = df1["name"].str.contains(name_tags) 
        result_name = df1.loc[df1 == True].to_frame().index.to_list()
        df2 = df[["tags","id"]].copy()
        df2 = df2.set_index("id")
        df2 = df2["tags"].str.contains(name_tags) 
        result_tags = df2.loc[df2 == True].to_frame().index.to_list()
        result = list(set(result_name + result_tags))
    if tags is not None:
        df1 = df[["tags","id"]].copy()
        df1 = df1.set_index("id")
        df1 = df1["tags"].str.contains(tags) 
        result = df1.loc[df1 == True].to_frame().index.to_list()
    return {'recipes': result}, 200

In [6]:
@app.route('/api/Ajouter_note', methods = ['POST'])
def get_ajouter_note():
    args = request.args
    id_user = args.get("id_user")
    id_recipes = args.get("id_recipes")
    note = 5
    df = pd.read_csv("../src/recipesDB/note.csv")
    data = {'user_id':id_user, 'recipe_id':id_recipes, 'rating':note}
    df = df.append(data, ignore_index=True)
    df.to_csv("../src/recipesDB/note.csv", index=False)
    return {'status': "j'aime"}, 200

In [7]:
# @app.route('/api/recette', methods = ['GET'])
# def get_recette_all():
#     args = request.args
#     id = args.get("id")
#     result = []
#     if id is None:
#         df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
#         id = list(df['id'])
#         df1 = []
#         for i in range(693):
#             """recipe = script.SmallRecipe(int(i))
#             df1.append(recipe.__dict__())"""
#             print(id[i], i)
#             response = requests.get('http://www.food.com/recipe/' + str(i))
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.text, 'html.parser')
#                 suggestion = script.MediumRecipe(int(id[i]), soup)
#                 df1.append(suggestion.__dict__())
#         return {'recipes': df1}, 200
#     recipe = script.SmallRecipe(int(id))
#     return {'recipes': recipe.__dict__()}, 200

In [10]:
if __name__ == '__main__':
    app.run(host='192.168.1.74', port=5050, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://192.168.1.74:5000/ (Press CTRL+C to quit)
192.168.1.74 - - [06/Jul/2022 16:57:22] "GET /api/recherche?search_button=chocolate HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:24] "GET /api/recherche?search_bar=chocolat HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:25] "GET /api/recherche?search_bar=chocolate HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:41] "GET /api/recherche?search_bar=chocolat HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:42] "GET /api/recherche?search_bar= HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:44] "GET /api/recherche?search_bar=C HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:50] "GET /api/recherche?search_bar= HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:53] "GET /api/recherche?search_bar=c HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:55] "GET /api/recherche?search_bar=ch HTTP/1.1" 200 -
192.168.1.28 - - [06/Jul/2022 16:57:56] "GET /api/recherche?search_bar=choc HTTP/1.1" 200 -
192.168.1.28